In [14]:
"""
SMS Detection FEMALE

Automatic phonation disease detection
"""
from __future__ import division, print_function, absolute_import
import os
import random
import numpy as np
import sklearn.preprocessing as prep
import librosa
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Input,Dense, Dropout
from tensorflow.keras.layers import Embedding, Bidirectional,Conv1D,GlobalMaxPooling1D
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import utils,losses
print("tf=",tf.__version__)
print("keras=",keras.__version__)

tf= 2.3.0
keras= 2.4.0


In [15]:
print("Imports doned")
def wav2mfcc(file_path, max_pad_len=30):
    wave, sampleRate = librosa.load(file_path, mono=True, sr=None)
    wave=prep.maxabs_scale(wave) #Normalize amplitude to +-1
    mfcc = librosa.feature.mfcc(wave, sr=sampleRate).transpose()
    #print(sampleRate,"wave=",wave.shape,"mfcc=",mfcc.shape,"file=",file_path)
    #print(mfcc[1,:5])
    dif=max_pad_len-mfcc.shape[0]
    #padding sample
    if dif>0:
        #fill holes
        #print("filling holes")
        for i in range(dif):
            mfcc=np.vstack([mfcc,np.zeros(mfcc.shape[1])])
    else:
        #take away excess
        #print("excess")
        mfcc=mfcc[:max_pad_len]
    #print(mfcc.shape)
    #print(mfcc[199,:5])
    return mfcc

#PATH_DATA="/home/francisco/nas/datasets/sonido/voz/BBDD_Smith-Magenis_Relabeling/asr_sms_21070201"
PATH_DATA="/home/spika/datasets/sonido/voz/asr_sms_f_21070801"
# load wav files with this format speaker_word_number.wav
class EasyASR(object):
    def __init__(self, filePath=PATH_DATA):
        '''
        Constructor
        '''
        self.max_pad_len=44
        self.nval=10
        basepath=filePath
        self.mfccs_n=[]
        self.speakers_n=[]
        self.diseases_n=[]
        self.ages_n=[]
        self.mfccs_d=[]
        self.speakers_d=[]
        self.diseases_d=[]
        self.ages_d=[]
        fNames=os.listdir(basepath)
        print("fNames=",len(fNames))
        random.shuffle(fNames)
        for i,fn in enumerate(fNames):
            #file name format speaker_word_number.wav
            n,_=fn.split(".")#take away extension .wav
            #decode file name format speaker-desease-age-number.wav
            #print("n=",n)
            speaker,disease,age,number=n.split("-")
            age=int(age)
            if disease!="SMS": continue
            mfccs=wav2mfcc(basepath+'/{}.wav'.format(n),self.max_pad_len)
            self.mfccs_d.append(mfccs)
            self.speakers_d.append(speaker)
            self.diseases_d.append(disease)
            self.ages_d.append(age)
        nSMS=len(self.mfccs_d)
        print("nSMS=",nSMS)
        random.shuffle(fNames)
        for i,fn in enumerate(fNames):
            #file name format speaker_word_number.wav
            n,_=fn.split(".")#take away extension .wav
            #decode file name format speaker-desease-age-number.wav
            #print("n=",n)
            speaker,disease,age,number=n.split("-")
            age=int(age)
            if disease!="N": continue
            mfccs=wav2mfcc(basepath+'/{}.wav'.format(n),self.max_pad_len)
            self.mfccs_n.append(mfccs)
            self.speakers_n.append(speaker)
            self.diseases_n.append(disease)
            self.ages_n.append(age)
            idxSMS=random.randint(self.nval,nSMS)
            self.mfccs_d.append(self.mfccs_d[idxSMS])
            self.speakers_d.append(self.speakers_d[idxSMS])
            self.diseases_d.append(self.diseases_d[idxSMS])
            self.ages_d.append(self.ages_d[idxSMS])

        self.mfccs_val   =self.mfccs_d   [:self.nval]+self.mfccs_n[:self.nval]
        self.speakers_val=self.speakers_d[:self.nval]+self.speakers_n[:self.nval]
        self.diseases_val=self.diseases_d[:self.nval]+self.diseases_n[:self.nval]
        self.ages_val    =self.ages_d    [:self.nval]+self.ages_n[:self.nval]

        self.mfccs   =self.mfccs_d   [nSMS+self.nval:]+self.mfccs_n[self.nval:]
        self.speakers=self.speakers_d[nSMS+self.nval:]+self.speakers_n[self.nval:]
        self.diseases=self.diseases_d[nSMS+self.nval:]+self.diseases_n[self.nval:]
        self.ages    =self.ages_d    [nSMS+self.nval:]+self.ages_n[self.nval:]

        self.leSpeakers=prep.LabelEncoder()
        self.leSpeakers.fit(self.speakers+self.speakers_val)
        self.iSpeakerLabels=self.leSpeakers.transform(self.speakers)
        self.iSpeakerLabels_val=self.leSpeakers.transform(self.speakers_val)
        self.leDiseases=prep.LabelEncoder()
        self.leDiseases.fit(self.diseases+self.diseases_val)
        self.iDiseaseLabels=self.leDiseases.transform(self.diseases)
        self.iDiseaseLabels_val=self.leDiseases.transform(self.diseases_val)
        self.leAges=prep.LabelEncoder()
        self.leAges.fit(self.ages+self.ages_val)
        self.iAgeLabels=self.leAges.transform(self.ages)
        self.iAgeLabels_val=self.leAges.transform(self.ages_val)
        print("Samples=",len(self.diseases))
        print("SMS=",self.diseases.count('SMS'))
        print("N   =",self.diseases.count('N'))
        print(set(self.diseases))
        print(set(self.ages))
        print(set(self.speakers))

ea=EasyASR()
print(ea.leDiseases)

Imports doned
fNames= 647
nSMS= 42
Samples= 1190
SMS= 595
N   = 595
{'SMS', 'N'}
{3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 17, 18}
{'SPS', 'MSC', 'SCL', 'PLC', 'RLL', 'LPM', 'BCA', 'MSP', 'RZA', 'LJL', 'VML', 'MRL', 'PFA', 'RMM', '11P', 'MRC', 'PPC', 'GRM', 'MMC', 'RBB', 'MSE', 'DDR', 'SNE', 'MXS', 'SOS', 'MCA', 'MLP', 'VGA', 'GMA', 'ZSD', 'RME', 'GRA', 'GSP', 'NGR', 'CRE', 'CCC', 'GOM', 'RMA', 'TMR', 'XMA', 'PRC', 'CMN', 'VRP', 'TPA', 'MTA', 'MAE', 'FCL', 'CMA', 'CCN', '10L', 'PLL', 'MML', 'BBA', 'RPJ', 'AIO', 'CSA', 'LEA', 'HJS', 'YZS', 'MVD', 'GES', 'MIR', 'LSC', 'VVV', 'MCS', 'PSE', 'TOA', 'PTA', 'CGC', 'SWA', 'RCE', 'SRL', 'MMJ', 'CRC', 'SFL', 'ERA', 'RFM', '10S', 'VPB', 'hMA', 'GLA', 'ITA', 'CAL', 'SRC', 'TRL', 'RRS', 'STA', 'ELL', 'ABN', 'PGT', 'DLA', 'YMA', 'rRA', 'RCL', 'PMA', 'RZV', 'CMS', 'DRS', 'YHJ', 'PML', 'RSL', 'CPA', 'CRI', 'BTS', 'PCP', 'FOV', 'CPM', 'RDA', 'SCI', 'PGA', 'GSA', 'HMF', 'HGC', 'GPC', 'RDI', 'OSS', 'MLC', 'CFE', 'SBA', 'IHI', 'MMF', 'MMN', 'BPC', 'APM', 'FDN', '

In [16]:
trainX, trainYdiseases, trainYspeakers, trainYages = np.array(ea.mfccs),np.array(ea.iDiseaseLabels),np.array(ea.iSpeakerLabels),np.array(ea.iAgeLabels)
#testX, testY = test
valX, valYdiseases, valYspeakers, valYages = np.array(ea.mfccs_val),np.array(ea.iDiseaseLabels_val),np.array(ea.iSpeakerLabels_val),np.array(ea.iAgeLabels_val)

maxlen=trainX.shape[1]
lstm_hidden_units=128
input_dim=trainX.shape[2]
outputDiseases_dim=len(ea.leDiseases.classes_)
outputSpeakers_dim=len(ea.leSpeakers.classes_)
outputAges_dim=len(ea.leAges.classes_)

print(outputDiseases_dim,outputSpeakers_dim,outputAges_dim)

# Converting labels to binary vectors
trainYdiseases_cat = utils.to_categorical(trainYdiseases,outputDiseases_dim)
trainYspeakers_cat = utils.to_categorical(trainYspeakers,outputSpeakers_dim)
trainYages_cat     = utils.to_categorical(trainYages    ,outputAges_dim)
print (trainX.shape,trainYdiseases_cat.shape,trainYspeakers_cat.shape,trainYages_cat.shape)

# Converting labels to binary vectors
valYdiseases_cat = utils.to_categorical(valYdiseases,outputDiseases_dim)
valYspeakers_cat = utils.to_categorical(valYspeakers,outputSpeakers_dim)
valYages_cat     = utils.to_categorical(valYages    ,outputAges_dim)
print (valX.shape,valYdiseases_cat.shape,valYspeakers_cat.shape,valYages_cat.shape)

2 130 12
(1190, 44, 20) (1190, 2) (1190, 130) (1190, 12)
(20, 44, 20) (20, 2) (20, 130) (20, 12)


In [17]:
# Network building
def build_model_asr_conv1d(input_dim,outputDiseases_dim,outputSpeakers_dim,outputAges_dim,hiden_dim=128):
    input = Input(shape=(ea.max_pad_len,input_dim))
    print("input=",input.shape)

    # Conv1D + global max pooling
    x = Conv1D(hiden_dim, 3, padding="valid", activation="relu", strides=1)(input)
    print(x.shape)
    x = Dropout(0.5)(x)
    x = Conv1D(hiden_dim, 3, padding="valid", activation="relu", strides=1)(x)
    print(x.shape)
    x = Dropout(0.5)(x)
    x = Conv1D(hiden_dim, 3, padding="valid", activation="relu", strides=1)(x)
    print(x.shape)
    #x = Dropout(0.5)(x)
    #x = Conv1D(hiden_dim, 3, padding="valid", activation="relu", strides=1)(x)
    x = GlobalMaxPooling1D()(x)
    print(x.shape)

    # We add a vanilla hidden layer:
    x = Dense(hiden_dim, activation="relu")(x)
    x = Dropout(0.5)(x)
    x = Dense(hiden_dim, activation="relu")(x)
    x = Dropout(0.5)(x)
    wd= Dense(hiden_dim, activation="relu")(x)
    sd= Dense(hiden_dim, activation="relu")(x)
    ad= Dense(hiden_dim, activation="relu")(x)
    # from: https://www.tensorflow.org/guide/keras/train_and_evaluate
    #       Passing data to multi-input, multi-output models
    prediction_diseases = Dense(outputDiseases_dim, activation="softmax", name="disease")(wd)
    prediction_speakers = Dense(outputSpeakers_dim, activation="softmax", name="speaker")(sd)
    prediction_ages     = Dense(outputAges_dim    , activation="softmax", name="age"    )(ad)

    model = Model(input, [prediction_diseases,prediction_speakers,prediction_ages])
    return model

In [18]:
model=build_model_asr_conv1d(input_dim,outputDiseases_dim,outputSpeakers_dim,outputAges_dim)
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

input= (None, 44, 20)
(None, 42, 128)
(None, 40, 128)
(None, 38, 128)
(None, 128)
Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 44, 20)]     0                                            
__________________________________________________________________________________________________
conv1d_3 (Conv1D)               (None, 42, 128)      7808        input_2[0][0]                    
__________________________________________________________________________________________________
dropout_4 (Dropout)             (None, 42, 128)      0           conv1d_3[0][0]                   
__________________________________________________________________________________________________
conv1d_4 (Conv1D)               (None, 40, 128)      49280       dropout_4[0][0]                  
_____

In [19]:
checkpoint_filepath="checkpoints/sms/"
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor='val_disease_accuracy',
    mode='max',
    save_best_only=True)

In [ ]:
# Training
history=model.fit(trainX, [trainYdiseases_cat,trainYspeakers_cat,trainYages_cat], batch_size=100,epochs=150,validation_data=(valX, [valYdiseases_cat,valYspeakers_cat,valYages_cat]),callbacks=[model_checkpoint_callback])

Epoch 1/150
12/12 [==============================] - ETA: 0s - loss: 51.7889 - disease_loss: 6.8215 - speaker_loss: 27.7637 - age_loss: 17.2037 - disease_accuracy: 0.4941 - speaker_accuracy: 0.0193 - age_accuracy: 0.1437WARNING:tensorflow:From /home/spika/anaconda3/envs/jax_gpu/lib/python3.8/site-packages/tensorflow/python/training/tracking/tracking.py:111: Model.state_updates (from tensorflow.python.keras.engine.training) is deprecated and will be removed in a future version.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: checkpoints/sms/assets
12/12 [==============================] - 2s 206ms/step - loss: 51.7889 - disease_loss: 6.8215 - speaker_loss: 27.7637 - age_loss: 17.2037 - disease_accuracy: 0.4941 - speaker_accuracy: 0.0193 - age_accuracy: 0.1437 - val_loss:

In [76]:
idxSample=288
print(trainX[idxSample:idxSample+1,:,:].shape)
predDisease,predSpeaker,predAge=model.predict(trainX[idxSample:idxSample+1,:,:])
print(predDisease,predAge)
y_predDisease= np.argmax(predDisease)
y_predSpeaker= np.argmax(predSpeaker)
y_predAge    = np.argmax(predAge)
print(y_predDisease,"=",np.max(predDisease))
print(y_predSpeaker,"=",np.max(predSpeaker))
print(y_predAge,"="    ,np.max(predAge))
disease=ea.leDiseases.inverse_transform([y_predDisease])
speaker=ea.leSpeakers.inverse_transform([y_predSpeaker])
age    =    ea.leAges.inverse_transform([y_predAge])
print("Predictions:")
print(speaker[0]," disease is \'",disease[0],"\' with age=",age[0])
print("Groundtruth:")
print("speaker=",ea.speakers[idxSample])
print("disease=",ea.diseases[idxSample])
print("ages   =",ea.ages[idxSample])
print()
print(ea.leDiseases.classes_)
print(ea.leSpeakers.classes_)
print(ea.leAges.classes_)

(1, 44, 20)
[[0.33287814 0.6671219 ]] [[5.40999286e-02 2.59878263e-02 6.59603626e-02 8.29324543e-01
  1.51112955e-02 8.31446052e-03 3.14634264e-04 3.54180338e-05
  1.01358157e-04 2.42827155e-04 1.21257195e-04 1.68156388e-04
  9.34792752e-06 3.33547396e-05 8.51068908e-05 1.73482440e-05
  1.77131114e-05 5.50124823e-05]]
1 = 0.6671219
243 = 0.78421324
3 = 0.82932454
Predictions:
VVV  disease is ' SMS ' with age= 6
Groundtruth:
speaker= VVV
disease= SMS
ages   = 6

['N' 'SMS']
['10L' '10S' '11C' '11P' '2JA' 'AAI' 'AAJ' 'ABN' 'ACD' 'ACI' 'ACR' 'AIE'
 'AIO' 'AJA' 'AMA' 'API' 'APM' 'ASC' 'AZM' 'BBA' 'BBR' 'BCA' 'BDC' 'BFA'
 'BHH' 'BPC' 'BTS' 'CAD' 'CAL' 'CAR' 'CCC' 'CCN' 'CFE' 'CGC' 'CII' 'CMA'
 'CMI' 'CMN' 'CMS' 'CPA' 'CPD' 'CPM' 'CRC' 'CRD' 'CRE' 'CRI' 'CRO' 'CSA'
 'CSR' 'DAP' 'DDR' 'DGR' 'DHD' 'DLA' 'DMD' 'DMH' 'DRS' 'ELL' 'EMA' 'ENF'
 'ERA' 'ERJ' 'FAF' 'FCD' 'FCL' 'FDN' 'FEA' 'FFR' 'FGI' 'FGJ' 'FLS' 'FMI'
 'FOV' 'FRM' 'GBR' 'GDT' 'GES' 'GLA' 'GMA' 'GMM' 'GOM' 'GPC' 'GRA' 'GRM'
 'GRR' 'GSA

In [84]:
idxSample=26
print(valX[idxSample:idxSample+1,:,:].shape)
predDisease,predSpeaker,predAge=model.predict(valX[idxSample:idxSample+1,:,:])
print(predDisease,predAge)
y_predDisease= np.argmax(predDisease)
y_predSpeaker= np.argmax(predSpeaker)
y_predAge    = np.argmax(predAge)
print(y_predDisease,"=",np.max(predDisease))
print(y_predSpeaker,"=",np.max(predSpeaker))
print(y_predAge,"="    ,np.max(predAge))
disease=ea.leDiseases.inverse_transform([y_predDisease])
speaker=ea.leSpeakers.inverse_transform([y_predSpeaker])
age    =    ea.leAges.inverse_transform([y_predAge])
print("Predictions:")
print(speaker[0]," disease is \'",disease[0],"\' with age=",age[0])
print("Groundtruth:")
print("speaker=",ea.speakers_val[idxSample])
print("disease=",ea.diseases_val[idxSample])
print("ages   =",ea.ages_val[idxSample])
print()
print(ea.leDiseases.classes_)
#print(ea.leSpeakers.classes_)
print(ea.leAges.classes_)

(1, 44, 20)
[[0.99868613 0.00131384]] [[0.00223355 0.00474983 0.0731962  0.20505498 0.17154486 0.18145065
  0.10715859 0.02997247 0.18782447 0.0261738  0.00175622 0.00135846
  0.00118167 0.00202186 0.0026162  0.00081778 0.00046647 0.00042197]]
0 = 0.99868613
116 = 0.039994366
3 = 0.20505498
Predictions:
MAA  disease is ' N ' with age= 6
Groundtruth:
speaker= FOV
disease= N
ages   = 11

['N' 'SMS']
[ 3  4  5  6  7  8  9 10 11 12 13 15 16 17 18 21 22 33]
